Setup:
Consider the 1-d boundary value problem from fluid dynamics:
$-u''(x)+V(x)u'(x)=f(x),x \in [0,1]$ <br>
$u(0) = u(1) = 1$ <br>
and we will set: <br>
V(x) = $\gamma$ <br>
f(x) = 1

Part 1: Problem formulation: <br>
a: write this as a variational problem: $A(u,\phi)=F(\phi)$
for some test function $\phi$ <br>


$A(u,\phi) = F(\phi)$



b: Take $\phi_i$ to be the hat functions discussed in class and aproximate u as a linear combination of these basis vectors: $u(x)=\sum_i u_i \phi_i(x)$. Show that part (a) becomes a linear algebra problem of the form Ax=b.
Derive expressions for the matrix A and bector b.
Show $A_ji = A(\phi_j,\phi_i)$ and can be writte as the sum of two matrices A = A<sub>1</sub> + A<sub>2</sub> where A<sub>1</sub> and A<sub>2</sub> corresopond to the first two terms on the left hand side of the ODE

c: Implement a routine that will return A as a sparse matrix and b given n and $\gamma$

In [2]:
import MatGenerator as mg

print(mg.generateA(4,1).todense())
print(mg.generateB(4))

[[10.  -4.5  0.   0. ]
 [-5.5 10.  -4.5  0. ]
 [ 0.  -5.5 10.  -4.5]
 [ 0.   0.  -5.5 10. ]]
[[0.2]
 [0.2]
 [0.2]
 [0.2]]


Part 2: implement GMRES algorithm
signature mygmres(l,b,x0,n,M,A)
returns solution of Ax=b with initial iterate x0
n is the dimension of the problem, A is nxn, M is nxn tha defines inner product use for calculateing vector norms (and error). Github, commits from all group members

Part 3: Solving the FEM problem <br>
use GMRES to solve for cases V(x) = 1 and V(x) = n+1 <br>
using M as the identity matrix. for each case, run with n=12,32,64,128 and l=2,4,8,16,32,64, increasing l until the error (norm of residual divided by n) is below 10<sup>-6</sup><br>
plot the most accurate solution (as a function of x) as well as the error vs functions of n and l

In [5]:
import MatGenerator as mg
A = mg.generateA(12,1.)
B = mg.generateB(12)
import GMRES as GM
from importlib import reload
reload(GM)
import numpy as np

alg = GM.gmres()
alg.mygmres(12,B, np.zeros((12,1)),12,np.eye(12),A)
err = alg.errors

import matplotlib.pyplot as plt
plt.figure()
plt.plot(err)


c:\Users\romrellb\Documents\phys530\LinAlgGroupHw\GMRES.py:44: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ym = np.linalg.solve(H,Beta*e1)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.